This converts `fingerprints.npy` to `.tsv` formatted t-SNE embeddings and plots of those embeddings in the `tsne/` and `plot/` folders respectively. If you add multiple values to `perplexity` and `initial_dims` then all combinations will be computed (in parallel). Good perplexities are in the range 1-200 with the best range around 30-100. Good `initial_dims` are in the range 30 and higher, with the dimensionality of your input data being the highest possible value (e.g., a 32x32 fingerprint would have a highest possible `initial_dims` value of 32x32=1024).

Change the "mode" to try different t-SNE variations.
* "fingerprints" will only use `fingerprints.npy`
* "predicted_labels" will only use `predicted_labels.npy`
* "predicted_encoding" will only use `predicted_encoding.npy`
* "combined" will use all of the above data

Things to do:
- Normalize audio
- Scale the spectrogram
- do all of the data cleaning so different factors between sample banks don't bias the feature extraction
- make sure to baseline this against traditional feature extraction 

In [6]:
%matplotlib inline
from matplotlib import pyplot as plt
from time import time
from utils import *
from os.path import join
from multiprocessing import Pool
import numpy as np
import itertools
import time as timeMod
from time import mktime
from datetime import datetime
import scipy.spatial
import scipy.spatial.distance as dist 
import pickle
from sklearn.decomposition import PCA

In [2]:
data_root = 'drumData/'
initial_dims = [30]
perplexities = [30]
mode = 'fingerprints'
drumNames = pickle.load(open(data_root+'drumNames.pickle'))
drumLengths = pickle.load(open(data_root+'drumLengths.pickle'))
colors = ['#000000', '#ff0000', '#00ff00', '#0000ff', '#ffff00', '#ff00ff', '#00ffff']
# mode = 'predicted_labels'
# mode = 'predicted_encoding'
# mode = 'combined'

# define TSNE calculation

In [3]:
X_2d_inspect = None

def save_data(data, fn):
    np.savetxt(fn, data, fmt='%.5f', delimiter='\t')

def savePlotsAndData(newData, data_root, prefix, colorMap, dataDir, plotDir, initial_dims=30, perplexity=30):    
    figsize = (16,16)
    pointsize = 30
    
    struct = timeMod.localtime(time())
    dt = datetime.fromtimestamp(mktime(struct))
    
    save_data(newData, join(data_root, dataDir+'/{}.{}.{}.2d - {}.tsv'.format(prefix, initial_dims, perplexity, dt)))
    
    plt.figure(figsize=figsize)
    plt.scatter(newData[:,0], newData[:,1], c=colorMap, s=pointsize)
    plt.tight_layout()
    plt.savefig(join(data_root, plotDir+'/{}.{}.{}_2D - {}.png'.format(prefix, initial_dims, perplexity, dt)))
    plt.close()
    
def tsne(data, data_root, prefix, colorMap, initial_dims=30, perplexity=30):
    mkdir_p(data_root + 'tsne')
    mkdir_p(data_root + 'plot')
    
    print initial_dims, perplexity, type(data), data.shape, data.dtype
    X_2d = list(bh_tsne(data, initial_dims=initial_dims, perplexity=perplexity, no_dims=2, verbose=True))
    X_2d = normalize(np.array(X_2d))
    
    savePlotsAndData(X_2d, data_root, prefix, colorMap, 'tsne', 'plot')
    
    return X_2d

def pca(data, data_root, prefix, colorMap):
    mkdir_p(data_root + 'pca')
    mkdir_p(data_root + 'pcaPlot')
    
    pcaInstance = PCA(n_components=2)  
    X_2d = pcaInstance.fit_transform(data)
    
    savePlotsAndData(X_2d, data_root, prefix, colorMap, 'pca', 'pcaPlot')
    
    return X_2d

    
def concatColors(segmentList, colorList):
    multiples = []
    #print segmentList, colorList
    for i in range(len(segmentList)):
        multiples.append([colorList[i]]*segmentList[i])
    return list(itertools.chain(*multiples))

### Load fingerprint data

In [4]:
# load and normalize any dataset we need
if mode == 'fingerprints' or mode == 'combined':
    drumPrints = []
    for drum in drumNames:
        drumPrints.append(np.load(join(data_root, drum+'_fingerprints.npy')))
        #print drum, drumPrints[-1].shape
    drumLengths = [drumPrint.shape[0] for drumPrint in drumPrints]
    colorMap = concatColors(drumLengths, colors)
    fingerprints = np.vstack(drumPrints)
    fingerprints = fingerprints.reshape(len(fingerprints), -1)
if mode == 'predicted_labels' or mode == 'combined':
    predicted_labels = np.load(join(data_root, 'predicted_labels.npy'))
    predicted_labels -= predicted_labels.min()
    predicted_labels /= predicted_labels.max()
if mode == 'predicted_encoding' or mode == 'combined':
    predicted_encoding = np.load(join(data_root, 'predicted_encoding.npy'))
    std = predicted_encoding.std(axis=0)
    predicted_encoding = predicted_encoding[:, std > 0] / std[std > 0]
    
if mode == 'fingerprints':
    data = fingerprints
if mode == 'predicted_labels':
    data = predicted_labels
if mode == 'predicted_encoding':
    data = predicted_encoding
if mode == 'combined':
    data = np.hstack((fingerprints, predicted_labels, predicted_encoding))

### Do dimensionality reduction

In [7]:
data = data.astype(np.float64)
colorMap = concatColors(drumLengths, colors)
useTsne = True
prefix = 'fingerprints'
def job(params):
    start = time()
    tsne2d = tsne(data, data_root, prefix, colorMap, initial_dims=params[0], perplexity=params[1])
    pca2d = pca(data, data_root, prefix, colorMap)
    print 'initial_dims={}, perplexity={}, {} seconds'.format(params[0], params[1], time() - start)
    return [tsne2d, pca2d]
params = list(itertools.product(initial_dims, perplexities))
pool = Pool()
dimReducedArrays = pool.map(job, params)
newData = dimReducedArrays[0]

30 30 <type 'numpy.ndarray'> (10560, 1024) float64
initial_dims=30, perplexity=30, 136.524546862 seconds


Process PoolWorker-9:
Process PoolWorker-13:
Process PoolWorker-14:
Process PoolWorker-10:
Process PoolWorker-16:
Process PoolWorker-15:
Process PoolWorker-11:
Process PoolWorker-12:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/Library/Frameworks/Python.fr

In [ ]:
len(dimReducedArrays)

# Metric calculation implementations

### Common setup 

In [8]:
# create point -> class hashmap
def getClassesPerSample(data):
    drumClasses = {}
    classIndex = 0
    for i in range(len(data)):
        if sum(drumLengths[0:classIndex+1]) <= i:
            classIndex += 1
        drumClasses[tuple(data[i])] = drumNames[classIndex]
    return drumClasses

drumClasses = getClassesPerSample(data)

numNeighbors = 10

## Arguments
## data -    an array of length n where array[i] is the fraction of neighbors of point i 
#            had the same class as point i.
# numPerClass - Assumes that points from the same class are contiguous and in
#            the same order as drumNames. Should use drumLengths for this parameter
# calcFunc - The summary statistic you want to calcuate per class (mean, medain, etc)
def calculateFuncPerClass(data, numPerClass, calcFunc):
    segments = [0]+numPerClass
    for i in range(1, len(segments)):
        segments[i] = segments[i] + segments[i-1]
    valuesPerClass = []
    for i in range(len(segments)-1):
        valuesPerClass.append(calcFunc(data[segments[i]:segments[i+1]]))
    return valuesPerClass


In [ ]:
import cPickle
cPickle.dump(classAccuracies_HD, open("1024DimClassAccuracies10neighbors.np", "w"))

### pairwise distance implementation

In [11]:
#using https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.spatial.distance.cdist.html
#and argpartition from - https://stackoverflow.com/questions/6910641/how-to-get-indices-of-n-maximum-values-in-a-numpy-array

## Arguments - pointData is m x n np array, with n points of m dimensions
# pointD numNeighbors is the number of nearest neighbors for which to compare classes
#
## returns - an array of length n where array[i] is the fraction of neighbors of point i 
#            had the same class as point i
def calculateKNNClassAccuracy(pointData, numNeighbors, printTimes=False):
    startTime = timeMod.time()
    pairwiseDist = dist.cdist(pointData, pointData)
    if printTimes:
        print "pairwise distances calculated", timeMod.time() - startTime
    kPartition = np.argpartition(pairwiseDist, -numNeighbors)
    if printTimes:
        print "partitions calculated", timeMod.time() - startTime
    fracSameNeighborClasses = np.zeros((len(data)))
    
    for i in range(len(pairwiseDist)):
        neighborIndexes = kPartition[i][-numNeighbors:]
        neighbors = [data[ind] for ind in neighborIndexes]
        
        sampleClass = drumClasses[tuple(data[i])]
        neighborClasses = [drumClasses[tuple(neighbor)] for neighbor in neighbors]
        numSameNeighborClasses = len(filter(lambda c : c == sampleClass, neighborClasses))
        fracSameNeighborClasses[i] = numSameNeighborClasses * 1.0 / numNeighbors
    
    if printTimes:
        print "knn classes calculated", timeMod.time() - startTime
        
    return fracSameNeighborClasses

classAccuracies_LD_TSNE = calculateKNNClassAccuracy(newData[0], numNeighbors, True)
classAccuracies_LD_PCA = calculateKNNClassAccuracy(newData[1], numNeighbors, True)
classAccuracies_HD = calculateKNNClassAccuracy(data, numNeighbors, True)

pairwise distances calculated 1.00153803825
partitions calculated 2.60620808601
knn classes calculated 11.9697771072
pairwise distances calculated 0.9041659832
partitions calculated 2.04635381699
knn classes calculated 10.9746088982
pairwise distances calculated 115.295583963
partitions calculated 116.938618898
knn classes calculated 125.641149998


In [15]:
print "mean for highDim, tsne, pca"
print round(np.mean(classAccuracies_HD), 3), round(np.mean(classAccuracies_LD_TSNE), 3), round(np.mean(classAccuracies_LD_PCA), 3)
print

print "std for highDim, tsne, pca"
print round(np.std(classAccuracies_HD), 3), round(np.std(classAccuracies_LD_TSNE), 3), round(np.std(classAccuracies_LD_PCA), 3)
print

print "mean per class for highDim"
print [round(n, 3) for n in calculateFuncPerClass(classAccuracies_HD, drumLengths, np.mean)]
print 

print "std per class for highDim"
print [round(n, 3) for n in calculateFuncPerClass(classAccuracies_LD_TSNE, drumLengths, np.std)]
print

print "mean per class for tsne"
print [round(n, 3) for n in calculateFuncPerClass(classAccuracies_LD_TSNE, drumLengths, np.mean)]
print 

print "std per class for tsne"
print [round(n, 3) for n in calculateFuncPerClass(classAccuracies_HD, drumLengths, np.std)]
print

print "mean per class for pca"
print [round(n, 3) for n in calculateFuncPerClass(classAccuracies_LD_PCA, drumLengths, np.mean)]
print 

print "std per class for pca"
print [round(n, 3) for n in calculateFuncPerClass(classAccuracies_LD_PCA, drumLengths, np.std)]
print

mean for highDim, tsne, pca
0.021 0.049 0.027

std for highDim, tsne, pca
0.066 0.197 0.087

mean per class for highDim
[0.0, 0.0, 0.0, 0.07, 0.033, 0.202, 0.112]

std per class for highDim
[0.272, 0.011, 0.076, 0.035, 0.0, 0.0, 0.011]

mean per class for tsne
[0.084, 0.001, 0.034, 0.003, 0.0, 0.0, 0.001]

std per class for tsne
[0.0, 0.0, 0.002, 0.046, 0.052, 0.17, 0.147]

mean per class for pca
[0.0, 0.0, 0.0, 0.092, 0.057, 0.141, 0.166]

std per class for pca
[0.0, 0.0, 0.0, 0.027, 0.05, 0.141, 0.248]



In [ ]:
a = np.array([[13,4,334,0 ,0 ,0],[0, 0, 0, 423,54444,63],[7333,85, 0, 0,0 ,539]])
np.argpartition(a, -2)

### kd-tree imlementtaion

In [ ]:
## Arguments - pointData is m x n np array, with n points of m dimensions
# pointD numNeighbors is the number of nearest neighbors for which to compare classes
#
## returns - an array of length n where array[i] is the fraction of neighbors of point i 
#            had the same class as point i
def calculateKNNClassAccuracy_kd(data, numNeighbors, printTiming=False, existingTree=None):
    startTime = 0
    if printTiming:
        print "building tree", timeMod.time()-startTime
    
    if existingTree is None:
        kdTree = scipy.spatial.KDTree(data) #takes about 2 minutes for 1024 dims
    else:
        kdTree = existingTree
    
    if printTiming:
        print "tree built", timeMod.time()-startTime 
        
    drumClasses = getClassesPerSample(data)
    
    return calcNeighborDistances(data, numNeighbors, kdTree, True)

#helper methods below

def calcNeighborDistances(pointData, numNeighbors, kdTree, printTiming=False):
    startTime = timeMod.time()
    if printTiming:
        print "calculating nearest neighbor classes"
    fracCorrectNeighbors = np.zeros((len(data)))
    for i in range(len(pointData)):
        if printTiming:
            if i % 1000 == 0:
                print "evaluating neighbors for point", i, timeMod.time() - startTime
        neighborDistances, neighborIndexes = kdTree.query(pointData[i], numNeighbors)
        neighbors = [data[ind] for ind in neighborIndexes]

        sampleClass = drumClasses[tuple(data[i])]
        neighborClasses = [drumClasses[tuple(neighbor)] for neighbor in neighbors]
        numCorrectNeighborClasses = len(filter(lambda c : c == sampleClass, neighborClasses))
        fracCorrectNeighbors[i] = numCorrectNeighborClasses * 1.0 / numNeighbors
        
    return fracCorrectNeighbors
    #from here do we want - mean, median, mode, stddev of correctClass fraction?
    #mean, median, mode, stddev of correctClass frac per class
    #% accuracy over a thresold?

def calculateFuncPerClass(data, numPerClass, calcFunc):
    segments = [0]+numPerClass
    for i in range(1, len(segments)):
        segments[i] = segments[i] + segments[i-1]
    valuesPerClass = []
    for i in range(len(segments)-1):
        valuesPerClass.append(calcFunc(data[segments[i]:segments[i+1]]))
    return valuesPerClass

kdTree1024Dim = scipy.spatial.KDTree(data)
kdTree2Dim = scipy.spatial.KDTree(newData)

classAccuracies_HD = calculateKNNClassAccuracy(data, numNeighbors, True, kdTree1024Dim)
classAccuracies_LD = calculateKNNClassAccuracy(newData, numNeighbors, True, kdTree2Dim)